In [1]:
%load_ext autoreload
%autoreload 2
%run ./initializer

Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [2]:
import polars as pl
from lgbm_classifier import LGBMClassifierModel
from lightgbm import LGBMClassifier

In [ ]:
train_df = pl.read_parquet("data/training_set_processed.parquet")
test_df = pl.read_parquet("data/test_set_datatype_processed.parquet")

In [4]:
lgbm_class_model = LGBMClassifierModel(
    objective="binary",        # binary click prediction
    boosting_type="gbdt",      # gradient‑boosted decision trees
    n_estimators=1000,         # upper cap on number of trees
    learning_rate=0.05,
)

In [5]:
X_train, y_train, X_val, y_val = lgbm_class_model.format_data(train_df)

In [6]:
lgbm_class_model.fit(
    X_train,
    y_train,
    X_val,
    y_val,
)

ValueError: could not convert string to float: '2013-04-04 08:32:15'